In [1]:
def create_permutation_matrix(s):
    s = list(s)
    def key(si):
        return si[0]
    s_p = [(si,i) for i,si in enumerate(s)]
    s_p.sort(key = key)
    l = len(s)
    M = MatrixSpace(ZZ,l)
    P = M([[0]*l]*l)
    for i in range(l):
        P[i,s_p[i][1]] = 1
    return P

In [2]:
def Basis(F0,s = None):
    m,n = F0.dimensions()
    if (s == None):
        s = [0]*m

    pi = create_permutation_matrix(s)
    P,L,U = (pi*F0).LU()
    r = U.rank()
    Lr = L[:r,:r]
    G = L[r:,:r]
    K = F0.base_ring()
    pR.<x> = K[]
    Pinv = P.inverse_of_unit()
    pi_inv = pi.inverse_of_unit()
    M = pi_inv * P *block_matrix(2,2,[[x,0],[-G*Lr.inverse(), 1]])*Pinv*pi
    MS = MatrixSpace(ZZ,m)
    u = pi_inv*P*(MS(Pinv)*pi*(vector(s)) + vector([1]*r+[0]*(m-r)))
    return M,u

In [3]:
m = 9
n = 4
F = GF(13)
pR.<x> = F[]

In [4]:
M = MatrixSpace(F,m,n)
MX = MatrixSpace(pR,m,n)

In [5]:
AX=MX.random_element()
A = M(AX.coefficient_matrix(0))

In [6]:
A

[ 0  8  2  8]
[ 5  1  7  1]
[ 0  8  0  3]
[ 0  0 10  0]
[ 3  0  9 12]
[ 0  0  0  2]
[11 11  0  4]
[ 0  0  0 12]
[10  4  4  9]

In [7]:
shifts = [ZZ.random_element(-4,5) for i in range(m)]
shifts

[-2, -4, -4, -2, 2, 3, -1, -2, -3]

In [8]:
B, new_shifts = Basis(A,shifts)

In [9]:
B.is_minimal_approximant_basis(A,1, shifts = shifts)

True

In [11]:
for i in range(100):
    AX=MX.random_element()
    A = M(AX.coefficient_matrix(0))
    shifts = [ZZ.random_element(-4,5) for i in range(m)]
    B, new_shifts = Basis(A,shifts)
    if(B.row_degrees(shifts = shifts) != new_shifts.list()):
        print('False',B.row_degrees(shifts = shifts),new_shifts.list())
        if (not B.is_minimal_approximant_basis(A,1, shifts = shifts)):
            print('Not a minimal approximant basis')

False [4, 4, 0, -2, 0, -1, 0, 3, 3] [4, 4, 0, 11, 0, 11, 0, 3, 3]
Not a minimal approximant basis
False [3, 4, 1, 3, 0, 4, 1, -2, 2] [3, 4, 1, 3, 11, 4, 1, 11, 2]
Not a minimal approximant basis
False [-2, -1, 4, -2, -2, 3, 4, 1, 3] [11, 12, 4, 11, 10, 3, 4, 1, 3]
Not a minimal approximant basis
False [4, 1, 4, -2, 4, 0, 0, 4, 1] [4, 1, 4, 11, 4, 12, 0, 4, 1]
Not a minimal approximant basis
False [-2, 3, 0, -1, -1, -2, 1, 3, 3] [10, 3, 0, 12, 12, 11, 1, 3, 3]
Not a minimal approximant basis


In [12]:
pR.<x> = GF(13)[]
m = 8
n = 5
sigma = 40
Ms = MatrixSpace(pR,m,n)
A = Ms.random_element()

In [24]:
def M_Basis(F,sigma,shift = None):
    pR = F.base_ring()
    x = pR.gen()
    m,n = F.dimensions()
    R = parent(F)
    if (shift == None):
        shift = [0]*m
        
    P = [0]*sigma
    u = [0]*sigma
    M = [0]*sigma
    P[0],u[0] = Basis(F.coefficient_matrix(0), shift)
    for k in range(1,sigma):
        F_prime = R(x^(-k)*P[k-1]*F).coefficient_matrix(0)
        M[k],u[k] = Basis(F_prime, u[k-1])
        P[k] = M[k]*P[k-1]
    return P, u, M

In [25]:
k = 0
for i in range(100):
    sigma = ZZ.random_element(1,50)
    shifts = [ZZ.random_element(-4,5) for i in range(m)]
    A = Ms.random_element()
    P,u, M = M_Basis(A,sigma, shift = shifts)
    if (not P[-1].is_minimal_approximant_basis(A,sigma, shifts = shifts)):
        print("Error")
        k += 1
k

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


97

In [27]:
P[-1].is_reduced(shifts = shifts)

False

In [28]:
M[1].row_degrees(shifts = u[0])

[12, 12, 12, 12, 12, 12, 12, 12]

In [29]:
u[0]

(4, 11, 0, 12, 12, 12, 12, 12)

In [35]:
M[1].d

[ 1 -1 -1 -1 -1 -1 -1 -1]
[-1  1 -1 -1 -1 -1 -1 -1]
[-1 -1  1 -1 -1 -1 -1 -1]
[-1 -1 -1  1 -1 -1 -1 -1]
[ 0  0  0  0  0 -1 -1 -1]
[-1 -1 -1 -1 -1  1 -1 -1]
[ 0  0  0  0 -1  0  0 -1]
[ 0  0  0  0 -1  0 -1  0]

In [ ]:
M[1].degree_matrix

In [ ]:
M[1].degree_matrix

In [31]:
u[0]

(4, 11, 0, 12, 12, 12, 12, 12)